In [10]:
import os
import json
import pandas as pd
import numpy as np
import gymnasium as gym
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecMonitor


## **User Define Class**

In [11]:
class Data_Class:

    # Internal Class
    class Trajectory_Class:
        def __init__(self, traj_series):
            self.traj_list = traj_series
            self.length = len(traj_series)

        def get_single_traj(self, index):
            return json.loads(self.traj_list[index])

    def __init__(self, path):
        self.path = path

        # 原始数据 Original Data
        self.trajs_prefer_list = []
        self.trajs_reject_list = []

        # 处理数据 Processed Data
        self.traj_prefer_list_list_tensor = []
        self.traj_reject_list_list_tensor = []

        # 启动函数 Start Function
        self.load_data(path)
        self.convert(self.trajs_prefer_list, self.traj_prefer_list_list_tensor)   # data convert 数据转换
        self.convert(self.trajs_reject_list, self.traj_reject_list_list_tensor)
        print("Data loaded successfully")

    def load_data(self, path):
        data = pd.read_csv(path)

        self.trajs_prefer_list = Data_Class.Trajectory_Class(data['preferred'])   # list data 数据
        self.trajs_reject_list = Data_Class.Trajectory_Class(data['rejected'])    # list data 数据

    def convert(self,
                list_json: Trajectory_Class,
                traj_list_list_tensor):

        # 获取第0条轨迹的第0时刻样本来确定维度
        # Get the first sample of the first trajectory to determine dimensions
        sample = list_json.get_single_traj(0)[0]
        state0 = np.array(sample['state'])
        action0 = np.array(sample['action'])

        # 获取 state action 维度
        # Get the dimensions of state and action
        self.dim_state = state0.size if state0.ndim == 0 else state0.shape[0]
        self.dim_action = action0.size if action0.ndim == 0 else action0.shape[0]

        # 数据批量转换 tensor
        # Convert data to tensor in batches
        for idx in range(list_json.length):
            traj = list_json.get_single_traj(idx)
            states, actions = [], []

            for time_i in traj:
                # 转换为 numpy，然后 torch tensor
                # Convert to numpy, then torch tensor
                state_np = np.array(time_i['state'])
                action_np = np.array(time_i['action'])

                state_t = torch.from_numpy(state_np).float()
                action_t = torch.from_numpy(action_np).float()

                # 如果是一维标量，要展开成长度1向量
                # If it's a one-dimensional scalar, expand it into a length 1 vector
                state_t = state_t.view(-1)
                action_t = action_t.view(-1)

                states.append(state_t)
                actions.append(action_t)

            # 将列表堆成张量 [L_i, dim]
            # Stack the list into a tensor [L_i, dim]
            states_tensor = torch.stack(states, dim=0)
            actions_tensor = torch.stack(actions, dim=0)

            # 将每条轨迹作为一个元组 (states, actions) 添加到列表中
            # Add each trajectory as a tuple (states, actions) to the list
            traj_list_list_tensor.append((states_tensor, actions_tensor))

# ——— 数据集与加载器 ———
# Dataset and DataLoader
class PreferenceDataset(Dataset):
    def __init__(self, pref, rej, gamma):
        assert len(pref) == len(rej)
        self.pref = pref
        self.rej = rej
        self.gamma = gamma

    def __len__(self):
        return len(self.pref)

    def __getitem__(self, idx):
        return (*self.pref[idx], *self.rej[idx])

# MLP 打分类 
# MLP scoring model class
class RewardMLP(nn.Module):
    def __init__(self, s_dim, a_dim, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            #0000FF # 这里在构造神经网络，后去可能需要调整神经网络结构 
            #0000FF # Here we construct the neural network, which may need to be adjusted later 
            nn.Linear(s_dim + a_dim, hidden_dim),  
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, s, a):
        # s: [L_i, s_dim], a: [L_i, a_dim]
        x = torch.cat([s, a], dim=-1)
        return self.net(x).squeeze(-1)



**check the total reward (in cartpole environment, total reward is the same as episode length) of preferred and rejected trajectory**

In [12]:
path = "trajectory_pairs.csv"
Data = Data_Class(path)

for i in range(10):
    print(i, len(Data.trajs_prefer_list.get_single_traj(i)), len(Data.trajs_reject_list.get_single_traj(i)))

Data loaded successfully
0 200 200
1 200 144
2 200 180
3 200 194
4 200 96
5 200 125
6 200 108
7 200 106
8 200 166
9 173 97


## **Reward Model Training**
use `trajectory_pairs.csv` to train reward model

In [13]:

# ———— Hyperparameters ————
gamma = 0.99      
lr = 1e-4          
batch_size = 16
num_epochs = 50

# ———— Load Data ————
path = "trajectory_pairs.csv"
Data = Data_Class(path)

# 自定义 collate_fn，保留变长序列
# Custom collate_fn to keep variable-length sequences
def variable_collate(batch):
    # batch: List of tuples (s_pref, a_pref, s_rej, a_rej)
    s_pf, a_pf, s_rj, a_rj = zip(*batch)
    return list(s_pf), list(a_pf), list(s_rj), list(a_rj)

# ——— Training Preperation ———
dataset = PreferenceDataset(
    Data.traj_prefer_list_list_tensor,
    Data.traj_reject_list_list_tensor,
    gamma
)
loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=variable_collate
)

reward_net = RewardMLP(Data.dim_state, Data.dim_action, hidden_dim=64) # 实例化 神经网络 MLP
optimizer  = optim.Adam(reward_net.parameters(), lr=lr)
loss_fn    = nn.BCEWithLogitsLoss()

# ——— Training Loop ———
for epoch in range(1, num_epochs + 1):
    total_loss = 0.0
    for s_pref_list, a_pref_list, s_rej_list, a_rej_list in loader:
        R_pref_batch = []
        R_rej_batch  = []

        # 计算 prefer 轨迹的回报
        # Calculate the return for preferred trajectories
        for s_pf, a_pf in zip(s_pref_list, a_pref_list):
            r_pf = reward_net(s_pf, a_pf)           # [L_i]
            discounts = torch.tensor([gamma**t for t in range(r_pf.size(0))], device=r_pf.device)
            R_pref_batch.append((r_pf * discounts).sum())

        # 计算 reject 轨迹的回报
        # Calculate the return for rejected trajectories
        for s_rj, a_rj in zip(s_rej_list, a_rej_list):
            r_rj = reward_net(s_rj, a_rj)          # [L_j]
            discounts = torch.tensor([gamma**t for t in range(r_rj.size(0))], device=r_rj.device)
            R_rej_batch.append((r_rj * discounts).sum())

        R_pref = torch.stack(R_pref_batch)
        R_rej = torch.stack(R_rej_batch)

        logits = R_pref - R_rej
        targets = torch.ones_like(logits)        
        loss = loss_fn(logits, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(R_pref_batch)

    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch}/{num_epochs} — Avg Loss: {avg_loss:.4f}")

# ——— Save Model ———
torch.save(reward_net.state_dict(), 'reward_net.pth')
print("🎉 Model has been saved to: reward_net.pth")


Data loaded successfully
Epoch 1/50 — Avg Loss: 1.0871
Epoch 2/50 — Avg Loss: 0.9674
Epoch 3/50 — Avg Loss: 0.8635
Epoch 4/50 — Avg Loss: 0.7858
Epoch 5/50 — Avg Loss: 0.7040
Epoch 6/50 — Avg Loss: 0.6431
Epoch 7/50 — Avg Loss: 0.5810
Epoch 8/50 — Avg Loss: 0.5335
Epoch 9/50 — Avg Loss: 0.4875
Epoch 10/50 — Avg Loss: 0.4552
Epoch 11/50 — Avg Loss: 0.4271
Epoch 12/50 — Avg Loss: 0.4042
Epoch 13/50 — Avg Loss: 0.3839
Epoch 14/50 — Avg Loss: 0.3686
Epoch 15/50 — Avg Loss: 0.3566
Epoch 16/50 — Avg Loss: 0.3452
Epoch 17/50 — Avg Loss: 0.3359
Epoch 18/50 — Avg Loss: 0.3289
Epoch 19/50 — Avg Loss: 0.3226
Epoch 20/50 — Avg Loss: 0.3161
Epoch 21/50 — Avg Loss: 0.3112
Epoch 22/50 — Avg Loss: 0.3068
Epoch 23/50 — Avg Loss: 0.3024
Epoch 24/50 — Avg Loss: 0.2987
Epoch 25/50 — Avg Loss: 0.2969
Epoch 26/50 — Avg Loss: 0.2935
Epoch 27/50 — Avg Loss: 0.2911
Epoch 28/50 — Avg Loss: 0.2891
Epoch 29/50 — Avg Loss: 0.2867
Epoch 30/50 — Avg Loss: 0.2839
Epoch 31/50 — Avg Loss: 0.2812
Epoch 32/50 — Avg Loss:

## **Reward Model Testing**
Load `trajectory_pairs.csv` to see if the total reward matches what the `MLP reward model` predicts

In [14]:
# ——— Load MLP Reward Model ———
reward_net_loaded = RewardMLP(Data.dim_state, Data.dim_action, hidden_dim=64)
reward_net_loaded.load_state_dict(torch.load('reward_net.pth', weights_only=True))
reward_net_loaded.eval()

# Load .csv Data
for i in range(10):
    traj_prefer_json = Data.trajs_prefer_list.get_single_traj(i)
    traj_reject_json = Data.trajs_reject_list.get_single_traj(i)

    # convert to tensor
    states_prefer  = torch.stack([torch.from_numpy(np.array(step['state'])).float().view(-1)
                                  for step in traj_prefer_json], dim=0)    # [L, s_dim]
    actions_prefer = torch.stack([torch.from_numpy(np.array(step['action'])).float().view(-1)
                                    for step in traj_prefer_json], dim=0)  # [L, a_dim]
    states_reject  = torch.stack([torch.from_numpy(np.array(step['state'])).float().view(-1)
                                    for step in traj_reject_json], dim=0)  # [L, s_dim]
    actions_reject = torch.stack([torch.from_numpy(np.array(step['action'])).float().view(-1)
                                    for step in traj_reject_json], dim=0)  # [L, a_dim]

    # 计算 prefer 轨迹的回报
    # Calculate the return for preferred trajectories
    with torch.no_grad():
        r_pref = reward_net_loaded(states_prefer, actions_prefer)          

    # 计算总回报
    # Calculate total return
    discounts = torch.tensor([gamma**t for t in range(r_pref.size(0))])
    total_return_prefer = (r_pref * discounts).sum()

    # 计算 reject 轨迹的回报
    # Calculate the return for rejected trajectories
    with torch.no_grad():
        r_rj = reward_net_loaded(states_reject, actions_reject)          

    # 计算总回报
    # Calculate total return
    discounts = torch.tensor([gamma**t for t in range(r_rj.size(0))])
    total_return_reject = (r_rj * discounts).sum()

    print(i, total_return_prefer, total_return_reject)



0 tensor(12.2832) tensor(11.5129)
1 tensor(11.5982) tensor(8.9034)
2 tensor(12.5000) tensor(10.9671)
3 tensor(12.3189) tensor(10.8672)
4 tensor(12.3967) tensor(6.5947)
5 tensor(12.2223) tensor(8.9502)
6 tensor(12.2360) tensor(6.0135)
7 tensor(12.1494) tensor(5.9533)
8 tensor(11.9423) tensor(11.1330)
9 tensor(11.5264) tensor(7.4788)


## **PPO-RLHF Training**

In [ ]:
# --------------------------------------------------------------------------------------------------
# 2. 定义一个 Wrapper，在 step 里用你的 MLP 计算 reward
# 2. Define a Wrapper that uses your MLP to calculate the reward in step
# --------------------------------------------------------------------------------------------------
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env, reward_model_path, device="cpu"):
        super().__init__(env)

        # state dimension
        self.dim_state = env.observation_space.shape[0]

        # action dimension
        try:                  self.dim_action = env.action_space.shape[0]
        except IndexError:    self.dim_action = 1

        self.device = device

        # Ensure the input dimensions match the checkpoint
        checkpoint = torch.load(reward_model_path, map_location=device, weights_only=False)
        input_dim = checkpoint['net.0.weight'].size(1)  # Extract input size from checkpoint
        self.reward_model = RewardMLP(input_dim - self.dim_action, self.dim_action).to(device)
        self.reward_model.load_state_dict(checkpoint)
        self.reward_model.load_state_dict(torch.load(reward_model_path, map_location=device, weights_only=False))
        self.reward_model.eval()
    
    def step(self, action):
        # 执行原 env，不用原 reward
        # Execute the original env, without the original reward
        obs, _, terminated, truncated, info = self.env.step(action)   
        
        state_tensor = torch.tensor(obs, dtype=torch.float32, device=self.device).unsqueeze(0)

        if isinstance(self.env.action_space, gym.spaces.Discrete):
            action_tensor = torch.tensor([action], dtype=torch.long, device=self.device)
        else:
            action_tensor = torch.tensor(action, dtype=torch.float32, device=self.device).unsqueeze(0)

        # 修改 action_tensor 形状
        # Modify action_tensor shape
        if action_tensor.ndim == 1:
            action_tensor = action_tensor.view(1, -1)

        # 计算奖励  
        # Calculate reward
        with torch.no_grad():
            reward_tensor = self.reward_model(state_tensor, 
                                              action_tensor)
        reward = reward_tensor.item()
        return obs, reward, terminated, truncated, info
    
def reset(self, **kwargs):
    return self.env.reset(**kwargs)


# --------------------------------------------------------------------------------------------------
# 3. 构造 vectorized 环境，并应用自定义 Wrapper
# 3. Construct vectorized environment and apply custom Wrapper
# --------------------------------------------------------------------------------------------------

# Log path
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_path = os.path.join("Training", current_time)
os.makedirs(log_path, exist_ok=True)

# Reward Model path
MODEL_PATH = "reward_net.pth"


#### Build env
vec_env = make_vec_env(
    env_id="CartPole-v1",
    n_envs=8,
    wrapper_class=lambda env: CustomRewardWrapper(env, MODEL_PATH, device="cpu"),
    monitor_dir=log_path
)

vec_env = VecMonitor(vec_env, log_path)


#### Build PPO Model
model = PPO(
    policy="MlpPolicy",
    env=vec_env,
    n_steps=256,
    device="cpu",
    verbose=1,
    tensorboard_log=log_path
)

#### Training
model.learn( 
    total_timesteps=50000,
    # callback=[eval_callback, save_callback]
)

#### Save Model
model.save(os.path.join(log_path, "model_full_training"))

Using cpu device
Logging to Training\2025-05-08_21-21-23\PPO_1


c:\Users\MATH-286-Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\vec_env\vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.7     |
|    ep_rew_mean     | 3.07     |
| time/              |          |
|    fps             | 3437     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 24.6        |
|    ep_rew_mean          | 3.7         |
| time/                   |             |
|    fps                  | 1770        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010768814 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.0244      |
|    learning_rate        | 0.

## **PPO-RLHF Testing**
You can find record video in `Training\2025-xx-xx\video`  
你可以在这个位置找到录像 `Training\2025-xx-xx\video`

In [ ]:
import os
import sys
cur_dir = os.getcwd()
pkg_dir = os.path.dirname(cur_dir)
if pkg_dir not in sys.path:
    sys.path.append(pkg_dir)
from PPO_Original import tools


# PPO-RLHF model testing
# log_path = "Training\\2025-05-08_19-59-36" #0000FF Change this to your log path
print("log_path:", log_path)
PPO_Model_Path = os.path.join(log_path, "model_full_training")
tools.test_model("PPO", PPO_Model_Path, n_episodes=2, render = True, record=True)

Training\2025-05-08_19-59-36\model_full_training



c:\Users\MATH-286-Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\EPFL\Course\MA2\[EE-568] Reinforcement Learning\EE-568-RL\PPO_RLHF\Training\2025-05-08_19-59-36\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Episode: 1 Score: 500.0
Episode: 2 Score: 500.0
